In [ ]:
# import all Imp Libraries
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
# punt, stopwords, wordnet
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# load the data
df=pd.read_csv('/content/IMDB_Dataset.csv', engine='python')
df

ParserError: unexpected end of data

#  Lowecasing :
    convert all the sentence words into lowercase

In [ ]:
df['Review']=df['Review'].str.lower()

# Remove Html tags
  <text>

In [ ]:
df[df['Review'].str.contains('<')]

,Review,sentiment
9,"superb product quality, very satisfied. <br>",positive
12,i hated every minute of it. 😡 <br>,negative
13,not worth the time or the price. <br>,negative
22,"terrible experience, would not recommend. <br>",negative
34,"terrible experience, would not recommend. <br>",negative
...,...,...
5970,i'm very disappointed with the service. <br>,negative
5973,"had a great time with friends, everything was ...",positive
5985,i absolutely loved this movie! 😊 <br>,positive
5987,"terrible experience, would not recommend. <br>",negative


In [ ]:
df['Review']=df['Review'].apply(lambda x: BeautifulSoup(x,'html.parser').get_text())

# Checking and Removing URLs


In [ ]:
df[df['Review'].str.contains('http')]

,Review,sentiment
5,"the service was excellent, highly recommend! h...",positive
14,"superb product quality, very satisfied. http:/...",positive
24,i hated every minute of it. 😡 http://example.com,negative
41,"superb product quality, very satisfied. http:/...",positive
50,not worth the time or the price. http://exampl...,negative
...,...,...
5959,i hated every minute of it. 😡 http://example.com,negative
5978,"had a great time with friends, everything was ...",positive
5980,i'm very disappointed with the service. http:/...,negative
5981,i hated every minute of it. 😡 http://example.com,negative


In [ ]:
z=lambda x:re.sub(r'http\S+','',x)
z('i hated every minute of it. 😡 http://example.com')

'i hated every minute of it. 😡 '

In [ ]:
df['Review']=df['Review'].apply(lambda x:re.sub(r'http\S+','',x))

In [ ]:
df[df['Review'].str.contains('http')]

,Review,sentiment


# Remove Punctuations


In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
df['Review']=df['Review'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

In [ ]:
df

,Review,sentiment
0,i absolutely loved this movie 😊 😍,positive
1,terrible experience would not recommend,negative
2,i absolutely loved this movie 😊 😍,positive
3,had a great time with friends everything was p...,positive
4,i absolutely loved this movie 😊,positive
...,...,...
5995,what a fantastic experience will come again,positive
5996,worst product ever total waste of money,negative
5997,had a great time with friends everything was p...,positive
5998,the service was excellent highly recommend,positive


# Chat word Tratment
    Replace slag/ Abbreviations (eg. 'u' ---> 'you')
    Convert informal  text to standard text

In [ ]:
chat_word_map={
    'u' : 'you',
    'ur' : 'your',
    'lol' : 'laugh out loud',
    'omg' : 'oh my god',
    'idk' : 'i do not know',
    'btw' : 'by the way',
    'brb' : 'be right back'
}

In [ ]:
def expand_chat_word(text):
  return ' '.join([chat_word_map[word] if word in chat_word_map else word for word in text.split()])

In [ ]:
df['Review']=df['Review'].apply(expand_chat_word)
df

,Review,sentiment
0,i absolutely loved this movie 😊 😍,positive
1,terrible experience would not recommend,negative
2,i absolutely loved this movie 😊 😍,positive
3,had a great time with friends everything was p...,positive
4,i absolutely loved this movie 😊,positive
...,...,...
5995,what a fantastic experience will come again,positive
5996,worst product ever total waste of money,negative
5997,had a great time with friends everything was p...,positive
5998,the service was excellent highly recommend,positive


# Spelling Correction (Optional But slow)
    correct misspelled words
    use tool >>> Textblob , SymSpell or autoCorrect

In [ ]:
from textblob import TextBlob
df['Review'].apply(lambda x : str(TextBlob(x).correct()))

,Review
0,i absolutely loved this movie 😊 😍
1,terrible experience would not recommend
2,i absolutely loved this movie 😊 😍
3,had a great time with friends everything was p...
4,i absolutely loved this movie 😊
...,...
5995,what a fantastic experience will come again
5996,worst product ever total waste of money
5997,had a great time with friends everything was p...
5998,the service was excellent highly recommend


# Stop word Remove
    Remove common word like 'The','is' ,'and '

In [ ]:
nltk.download('stopwords')
stop_word=set(stopwords.words('english'))
print(stop_word)

{'yourself', 'just', 'haven', "she'd", 'when', "you're", 'how', 'were', 've', 'as', 'for', 'you', 'mustn', 'by', 'who', "they've", 'some', 'them', 'itself', 'this', 'will', 'on', 'not', 'does', "she'll", 'then', 'because', 'aren', "won't", "they'd", 'both', 'few', 'mightn', "i've", 'we', 'me', 'too', 'any', "i'll", 'needn', 'having', "it's", 'wasn', 'weren', 'at', 'hasn', 'm', 'a', 'down', 'most', 'off', 'if', 'own', 'the', "weren't", 'below', "hasn't", "aren't", 'hers', 'more', "she's", 'myself', 'your', "we'll", "don't", 'that', 'about', 'yourselves', 'isn', 'their', 'are', 'over', 'while', 'is', 'there', "wasn't", 'which', 'theirs', 'than', 'between', "needn't", 'out', 'ma', 'been', 'herself', "isn't", 'ain', 'shan', "he'd", 'now', 'didn', "shan't", 'no', "we'd", 'had', 'ourselves', "i'd", 't', "should've", 'with', 'doing', 'ours', 'once', 'here', 'all', 'such', 'these', 'very', 'll', 'and', 'my', 'each', 'won', 'should', 'yours', 'why', 'during', 'other', 'can', 'whom', 'what', 'up

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
def remove_stop_words(text):
  tokens=word_tokenize(text)
  return ' '.join([word for word in tokens if word not in stop_word])

In [ ]:
import nltk
nltk.download('punkt_tab')
df['Review']=df['Review'].apply(remove_stop_words)
df

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Review,sentiment
0,absolutely loved movie 😊 😍,positive
1,terrible experience would recommend,negative
2,absolutely loved movie 😊 😍,positive
3,great time friends everything perfect laugh loud,positive
4,absolutely loved movie 😊,positive
...,...,...
5995,fantastic experience come,positive
5996,worst product ever total waste money,negative
5997,great time friends everything perfect,positive
5998,service excellent highly recommend,positive


# Handling emojis
    Convert emotion images into text
    

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.5 MB/s eta 0:00:00


In [ ]:
import emoji
emoji.demojize('😎',delimiters=(' ',' '))

' smiling_face_with_sunglasses '

In [ ]:
df['Review']=df['Review'].apply(lambda x: emoji.demojize(x,delimiters=(' ',' ')))
df

,Review,sentiment
0,absolutely loved movie smiling_face_with_smil...,positive
1,terrible experience would recommend,negative
2,absolutely loved movie smiling_face_with_smil...,positive
3,great time friends everything perfect laugh loud,positive
4,absolutely loved movie smiling_face_with_smil...,positive
...,...,...
5995,fantastic experience come,positive
5996,worst product ever total waste money,negative
5997,great time friends everything perfect,positive
5998,service excellent highly recommend,positive


# Tokenization :
    Split sentences into words
    

In [ ]:
df['Tokenize_Review']=df['Review'].apply(word_tokenize)
df

,Review,sentiment,Tokenize_Review
0,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s..."
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]"
2,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s..."
3,great time friends everything perfect laugh loud,positive,"[great, time, friends, everything, perfect, la..."
4,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s..."
...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]"


# Stemming :     
    Reduce words to base / root (e.g 'Playing ---> 'Play',
    played ---> 'Play')

In [ ]:
stemmer=PorterStemmer()
def steam_word(text):
  return [stemmer.stem(word) for word in text]
df['Clean_Review']=df['Tokenize_Review'].apply(steam_word)

In [ ]:
df

,Review,sentiment,Tokenize_Review,Clean_Review
0,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]","[terribl, experi, would, recommend]"
2,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
3,great time friends everything perfect laugh loud,positive,"[great, time, friends, everything, perfect, la...","[great, time, friend, everyth, perfect, laugh,..."
4,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
...,...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]","[fantast, experi, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]","[worst, product, ever, total, wast, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]","[great, time, friend, everyth, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]","[servic, excel, highli, recommend]"


# Lemmatization  :     
    Convert woirds to dict form e.g('better' ---->  'good')

In [ ]:
lemmatizer=WordNetLemmatizer()
def lemmatizer_word(text):
  return [lemmatizer.lemmatize(word) for word in text]

In [ ]:
df['Clean_Review']=df['Clean_Review'].apply(lemmatizer_word)
df

,Review,sentiment,Tokenize_Review,Clean_Review
0,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]","[terribl, experi, would, recommend]"
2,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
3,great time friends everything perfect laugh loud,positive,"[great, time, friends, everything, perfect, la...","[great, time, friend, everyth, perfect, laugh,..."
4,absolutely loved movie smiling_face_with_smil...,positive,"[absolutely, loved, movie, smiling_face_with_s...","[absolut, love, movi, smiling_face_with_smilin..."
...,...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]","[fantast, experi, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]","[worst, product, ever, total, wast, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]","[great, time, friend, everyth, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]","[servic, excel, highli, recommend]"


In [ ]:
# BOW : Vectorization
from sklearn.feature_extraction.text import CountVectorizer

df['string_review']=df['Clean_Review'].apply(lambda x :' '.join(x))
vectorizer=CountVectorizer()
bow=vectorizer.fit_transform(df['string_review'])
print(bow)
print(vectorizer.vocabulary_)
print(bow[4].toarray())

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 27457 stored elements and shape (6000, 39)>
  Coords	Values
  (0, 0)	1
  (0, 18)	1
  (0, 21)	1
  (0, 30)	1
  (0, 29)	1
  (0, 9)	1
  (1, 32)	1
  (1, 8)	1
  (1, 38)	1
  (1, 26)	1
  (2, 0)	1
  (2, 18)	1
  (2, 21)	1
  (2, 30)	1
  (2, 29)	1
  (2, 9)	1
  (3, 12)	1
  (3, 33)	1
  (3, 11)	1
  (3, 6)	1
  (3, 22)	1
  (3, 16)	1
  (3, 17)	1
  (4, 0)	1
  (4, 18)	1
  :	:
  (5994, 1)	1
  (5995, 8)	1
  (5995, 10)	1
  (5995, 1)	1
  (5996, 24)	1
  (5996, 36)	1
  (5996, 4)	1
  (5996, 34)	1
  (5996, 35)	1
  (5996, 20)	1
  (5997, 12)	1
  (5997, 33)	1
  (5997, 11)	1
  (5997, 6)	1
  (5997, 22)	1
  (5998, 26)	1
  (5998, 28)	1
  (5998, 7)	1
  (5998, 14)	1
  (5999, 24)	1
  (5999, 36)	1
  (5999, 4)	1
  (5999, 34)	1
  (5999, 35)	1
  (5999, 20)	1
{'absolut': 0, 'love': 18, 'movi': 21, 'smiling_face_with_smiling_ey': 30, 'smiling_face_with_heart': 29, 'ey': 9, 'terribl': 32, 'experi': 8, 'would': 38, 'recommend': 26, 'great': 12, 'time': 33, 'friend': 11, '

In [ ]:
print(bow[10].toarray())

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
  0 0 1]]


In [ ]:
bow.shape

(6000, 39)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(ngram_range=(1,2))
n_gram=vectorizer.fit_transform(df['string_review'])
n_gram[0].toarray()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]])

In [ ]:
n_gram.shape

(6000, 89)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x_tfidf=vectorizer.fit_transform(df['string_review'])
x_tfidf.shape

(6000, 39)

In [ ]:
x_tfidf[0].toarray()

array([[0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.42537028, 0.        ,
        0.        , 0.42537028, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [ ]:
from scipy.sparse import csr_matrix
x=x_tfidf
# store each row as a dense numpy array in a list
feature_list=[x[i].toarray().flatten() for i in range(x.shape[0])]
feature_list

[array([0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.42537028, 0.        ,
        0.        , 0.42537028, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.37164533,
        0.42537028, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.44215901, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.43667923, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.55398857, 0.        , 0.

In [ ]:
# or if you want to store as sparse vector (for storage efficiency)
sparse_feature_list=[x[i] for i in range(x.shape[0])]
sparse_feature_list

[<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 6 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 6 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 7 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 5 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 6 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements and shape (1, 39)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4 stored elements 

In [ ]:
from scipy.sparse import save_npz
save_npz('tfidf_Matrix.npz',x)     # why x -- x is our csr_matrix

In [ ]:
from scipy.sparse import load_npz
x=load_npz('tfidf_Matrix.npz')
x

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 27457 stored elements and shape (6000, 39)>

In [ ]:
y=df['sentiment']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
y

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
# import all Important libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Convert TFIDF Sparse Matrix To Array
x_array=x.toarray()
x_array

array([[0.42537028, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.55398857],
       [0.42537028, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.42194103, 0.        ,
        0.        ]])

In [ ]:
# Train test split
x_train,x_test,y_train,y_test=train_test_split(x_array,y,test_size=0.2,random_state=42)

In [ ]:
# train the model
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
# pred
y_pred=model.predict(x_test)
y_pred

array([1, 0, 1, ..., 1, 0, 1])

In [ ]:
accuracy_score(y_test,y_pred)

1.0

In [ ]:
y_pred=model.predict(x_train)
y_pred

array([0, 0, 0, ..., 1, 1, 0])

In [ ]:
accuracy_score(y_train,y_pred)

1.0

In [ ]:
y_pred_test=model.predict(x_test)
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       571
           1       1.00      1.00      1.00       629

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200

